# Chapter 3 Hydraulics: Weir and Notches

1. [Introduction: Weir and Notches](#1.-Introduction)
2. [Simulation](#2.-Simulation)
3. [Self-Assessment](#3.-Self-Assessment)

## 1. Introduction

### Interactive Weir Flow Estimator

This notebook provides an interactive tool to explore how different types of **sharp-crested weirs** behave under varying conditions. It calculates and visualizes the **discharge** \( Q \) as a function of upstream water head \( H \), using empirical equations and standard discharge coefficients $( C_d $).

---

### Types of Weirs Supported

The tool includes the following weir geometries:

| Weir Type     | Description                                                           |
|---------------|-----------------------------------------------------------------------|
| Rectangular   | A horizontal crest with uniform width, optionally contracted          |
| Triangular    | A V-notch weir with an included angle (e.g., 90°), often used at low flows |
| Cipolletti    | A trapezoidal weir with 1:4 side slopes, designed to self-compensate for end contractions |

Each weir formula relates discharge \( Q \) to the upstream head \( H \) as shown below:

- **Rectangular Weir**:  
  $$
  Q = \frac{2}{3} C_d \, b \, \sqrt{2g} \, H^{3/2}
  $$

- **Triangular (V-Notch) Weir**:  
  $$
  Q = \frac{8}{15} C_d \tan\left(\frac{\theta}{2}\right) \sqrt{2g} \, H^{5/2}
  $$

- **Cipolletti Weir**:  
  Same form as the rectangular weir, with a standard Cd for the self-compensating geometry.

Where:
- $( C_d $) = discharge coefficient (typically between 0.55 and 0.64)  
- $( b $) = crest width [m]  
- $( \theta $) = notch angle in degrees (for triangular weir)  
- $( H $) = head over crest [m]  
- $( g = 9.81 \text{ m/s}^2 $)

---

###  User Controls

- **Weir Type**: Select from Rectangular, Triangular, or Cipolletti
- **Crest Width $( b $)**: For rectangular/Cipolletti weirs
- **V-Notch Angle $( \theta $)**: For triangular weirs
- **Head Range $( H $)**: Maximum value for head to plot
- **Discharge Coefficient $( C_d $)**: Selected from recommended presets based on field or lab calibration

---

###  Output

The chart dynamically plots the relationship between head $( H $) and discharge $( Q $), allowing you to:

- Compare performance across weir types
- Visualize sensitivity of flow to small changes in head
- Explore how design geometry and material calibration affect flow capacity



## 2. Simulation

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Dropdown
g = 9.81  # m/s²

# Weir formulas
def rectangular_weir_Q(H, b, Cd):
    return (2 / 3) * Cd * b * np.sqrt(2 * g) * H**1.5

def triangular_weir_Q(H, theta_deg, Cd):
    theta = np.radians(theta_deg)
    return (8 / 15) * Cd * np.tan(theta / 2) * np.sqrt(2 * g) * H**(5/2)

def cipolletti_weir_Q(H, b, Cd):
    return (2 / 3) * Cd * b * np.sqrt(2 * g) * H**1.5

# Cd presets
cd_presets = {
    "Rectangular": {
        "Standard (0.60)": 0.60,
        "Lab Calibrated (0.62)": 0.62,
        "Field Rough (0.58)": 0.58
    },
    "Triangular": {
        "Standard (0.58)": 0.58,
        "Sharp-Crested (0.60)": 0.60,
        "Field Calibrated (0.55)": 0.55
    },
    "Cipolletti": {
        "Standard (0.62)": 0.62,
        "High-Precision (0.64)": 0.64,
        "Rounded Crest (0.60)": 0.60
    }
}

# Interactive plotting logic
def plot_weir(weir_type, cd_label, b, theta, H_max):
    Cd = cd_presets[weir_type][cd_label]
    H_vals = np.linspace(0.01, H_max, 300)

    if weir_type == "Rectangular":
        Q_vals = rectangular_weir_Q(H_vals, b, Cd)
        label = f"Rectangular (b = {b:.2f} m, Cd = {Cd})"
    elif weir_type == "Triangular":
        Q_vals = triangular_weir_Q(H_vals, theta, Cd)
        label = f"Triangular (θ = {theta:.0f}°, Cd = {Cd})"
    elif weir_type == "Cipolletti":
        Q_vals = cipolletti_weir_Q(H_vals, b, Cd)
        label = f"Cipolletti (b = {b:.2f} m, Cd = {Cd})"

    plt.figure(figsize=(8, 5))
    plt.plot(H_vals, Q_vals, color='teal', linewidth=2, label=label)
    plt.xlabel("Head H (m)")
    plt.ylabel("Discharge Q (m³/s)")
    plt.title(f"Discharge vs Head – {weir_type} Weir")
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.legend()
    plt.tight_layout()
    plt.show()

# Sync dropdown selection with Cd
def interactive_plot(weir_type, b, theta, H_max):
    cd_choices = list(cd_presets[weir_type].keys())
    
    def plot_with_cd(cd_label):
        plot_weir(weir_type, cd_label, b, theta, H_max)
    
    interact(
        plot_with_cd,
        cd_label=Dropdown(options=cd_choices, value=cd_choices[0], description="Cd")
    )

# Outer interact
interact(
    interactive_plot,
    weir_type=Dropdown(options=["Rectangular", "Triangular", "Cipolletti"], description="Weir Type"),
    b=FloatSlider(value=1.0, min=0.1, max=5.0, step=0.1, description="Width b (m)"),
    theta=FloatSlider(value=90, min=30, max=120, step=1, description="V-Notch θ (°)"),
    H_max=FloatSlider(value=1.0, min=0.2, max=2.0, step=0.05, description="Max Head H (m)")
)

interactive(children=(Dropdown(description='Weir Type', options=('Rectangular', 'Triangular', 'Cipolletti'), v…

<function __main__.interactive_plot(weir_type, b, theta, H_max)>

## 3. Self-Assessment

### Conceptual Questions
- Why does the discharge of a weir increase with head in a nonlinear manner? How is this reflected in the exponent of the discharge equation?
- How does the shape of the weir (rectangular vs triangular vs Cipolletti) influence its sensitivity to flow depth (H)?
- What role does the discharge coefficient C_d play in the weir equation, and why isn’t it constant across all field conditions?
- Why are V-notch (triangular) weirs often preferred for measuring small discharges, especially in laboratory and environmental monitoring setups?
- How does the geometry of the Cipolletti weir compensate for end contractions present in a standard rectangular weir?

### Reflective Questions
- When comparing two weirs of equal crest width and head, which one typically yields higher discharge: a standard rectangular or a Cipolletti weir? Why?
- If you increase the notch angle of a triangular weir from 60° to 120°, how would that affect the sensitivity of flow to head changes?
- What practical constraints might limit the maximum head that can be used in a weir structure in the field?
- Imagine you're installing a weir in a remote stream to measure stormwater runoff. Which geometry would you choose and why?
- How might sediment accumulation or debris at the crest of a weir affect the validity of the weir discharge equation?

✅ Quiz Questions
Q1. The discharge equation for a triangular (V-notch) weir includes which head exponent?
- 1.5
- 2
- 2.5
- 3
🟢 Correct Answer: C

Q2. The discharge coefficient C_d accounts for:
- Pipe friction losses
- Gravity changes
- Real-world effects like viscosity, turbulence, and shape
- Unit conversion from feet to meters
🟢 Correct Answer: C

Q3. A Cipolletti weir is designed to:
- Measure industrial pipe flows
- Eliminate the need for head measurement
- Increase surface roughness
- Offset end contractions in rectangular weirs
🟢 Correct Answer: D

Q4. If all other parameters are constant, increasing C_d will:
- Increase computed flow
- Decrease computed flow
- Have no effect
- Reverse flow direction
🟢 Correct Answer: A

Q5. Which of the following is most appropriate for measuring small, variable flows in environmental monitoring?
- Broad-crested weir
- V-notch (triangular) weir
- Rectangular weir with wide crest
- Compound ogee spillway
🟢 Correct Answer: B